In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
!pip install pyreadr

!pip install pyGPGO

In [12]:
import pandas as pd
import numpy as np
import functools
from matplotlib import pyplot as plt
import datetime
import dill
import seaborn as sns
from collections import Counter
import statsmodels.api as sm
from statsmodels.graphics.gofplots import qqplot
import pylab
from datetime import datetime
import pyreadr
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import boxcox
from math import log
from math import exp

# pyGPGO

from pyGPGO.covfunc import squaredExponential
from pyGPGO.surrogates.GaussianProcess import GaussianProcess
from pyGPGO.surrogates.RandomForest import RandomForest
from pyGPGO.GPGO import GPGO
from pyGPGO.acquisition import Acquisition

# Keras
import tensorflow as tf
import keras
from keras.layers import Dense, Dropout, BatchNormalization,LSTM
from keras.models import Sequential
from keras.models import load_model
from keras.callbacks import EarlyStopping,ReduceLROnPlateau
from keras import backend as K

## Caricamento dataset

In [8]:
time_series = pd.read_csv('/content/drive/My Drive/Streaming Data Management and time series/time_series_dataset.csv',
                          sep=';')

In [ ]:
time_series.head(10)

In [ ]:
time_series['Year'] = time_series['Data'].apply(lambda x: int(x.split('-')[0]))

In [ ]:
np.random.seed(2020)

### Preprocessing 

In [ ]:
look_back = 365
future=1

In [ ]:
def crea_tensore_ml(array,look_back,future):
  x=list()
  y=list()
  valori =array
  inizio=0
  fine=look_back+future
  while inizio+fine<=len(valori):
    sequenza=valori[inizio:(inizio)+fine]
    x.append(sequenza[:(len(sequenza)-future)])
    y.append(sequenza[(len(sequenza)-future):])
    inizio=inizio+1
  return np.array(x), np.array(y)

In [ ]:
train_originale = time_series[time_series['Year']<=2016]

validation_originale = time_series[time_series['Year']==2017]

test_originale = time_series[time_series['Year']==2018]

train_ultimi365 = train_originale[len(train_originale)-365:]
train_ultimi365_val = pd.concat([train_ultimi365, validation_originale], axis=0)

ultimi365 = validation_originale
test_ultimi365_val = pd.concat([ultimi365,test_originale],axis=0)

test_def = test_originale

In [ ]:
train_originale.head(3)

In [ ]:
train_ultimi365_val.head(3)

train_ultimi365_val.reset_index(drop=True,inplace=True)

test_ultimi365_val.reset_index(drop=True,inplace=True)

test_ultimi365_val.head(3)

In [ ]:
def create_dataset(dataset, look_back):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)

#### Standard Scaler

In [ ]:
dati_norm = train_originale['value'].values

dati_norm = np.reshape(dati_norm,(-1,1))

In [ ]:
#scaler = MinMaxScaler(feature_range=(0, 1))

scaler = StandardScaler()
prova = scaler.fit_transform(dati_norm) #standardizzazione su tutto il dataset

In [ ]:
train_originale_dati = np.reshape(np.array(train_originale['value'].dropna()),(train_originale.shape[0],1))
train_ultimi365_val_dati = np.reshape(np.array(train_ultimi365_val['value'].dropna()),(train_ultimi365_val.shape[0],1))

test_ultimi365_val_dati = np.reshape(np.array(test_ultimi365_val['value'].dropna()),(test_ultimi365_val.shape[0],1))

test_def_dati = np.reshape(np.array(test_def['value'].dropna()),(test_def.shape[0],1))

In [ ]:
train_originale_dati_norm = scaler.transform(train_originale_dati)

train_ultimi365_val_dati_norm = scaler.transform(train_ultimi365_val_dati)

test_ultimi365_val_dati_norm = scaler.transform(test_ultimi365_val_dati)

test_def_dati_norm = scaler.transform(test_def_dati)

In [ ]:
trainX, trainY = create_dataset(train_originale_dati_norm, look_back)

validationX, validationY = create_dataset(train_ultimi365_val_dati_norm, look_back)

testX, testY = create_dataset(test_ultimi365_val_dati_norm, look_back)

In [ ]:
testX_def = np.reshape(test_def_dati_norm, (1,test_def_dati_norm.shape[0]))

In [ ]:
trainX.shape, trainY.shape, validationX.shape, validationY.shape, testX.shape, testY.shape, testX_def.shape

In [ ]:
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
validationX = np.reshape(validationX, (validationX.shape[0], 1, validationX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

testX_def = np.reshape(testX_def, (testX_def.shape[0], 1, testX_def.shape[1]))

In [ ]:
trainX.shape, validationX.shape, testX.shape, testX_def.shape

In [ ]:
trainY = np.reshape(trainY, (trainY.shape[0],1))

validationY = np.reshape(validationY, (validationY.shape[0],1))

testY = np.reshape(testY, (testY.shape[0],1))

In [ ]:
trainY.shape, validationY.shape, testY.shape

##  Creazione LSTM

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
model = Sequential()

model.add(LSTM(364, activation='tanh',
               kernel_regularizer=keras.regularizers.l2(0.001),
               recurrent_regularizer=keras.regularizers.l2(0.001),
               input_shape=(1,trainX.shape[2])))

model.add(Dense(737,activation='relu'))
model.add(Dropout(0.24))
model.add(Dense(954,activation='relu'))
model.add(Dropout(0.24))
model.add(Dense(1,activation='linear'))

model.summary()

In [ ]:
rms = keras.optimizers.RMSprop(lr = 0.0009)

model.compile(optimizer= rms,
                loss='mse')

In [ ]:
es = EarlyStopping(monitor = 'val_loss',
                   mode = 'min',
                   patience = 10,
                   restore_best_weights = False)


network_history = model.fit(trainX,trainY,shuffle=False, validation_data=(validationX,validationY),
                            epochs=100,batch_size=128,callbacks=[es])

In [ ]:
epoch_completed=list(range(1,len(network_history.epoch)+1))
plt.figure()
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.plot(epoch_completed, network_history.history['loss'])
plt.plot(epoch_completed, network_history.history['val_loss'])
plt.legend(['Training', 'Validation'])

In [ ]:
model.save('/content/drive/My Drive/Streaming Data Management and time series/modelli_salvati/1passo/model_best_2_hidden_layer.h5')

In [ ]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

In [ ]:
testPredict.shape, testX.shape

In [ ]:
# ritorna dato formato originale
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

In [ ]:
trainY[0].shape

In [ ]:
trainPredict[:,0].shape

In [ ]:
trainY_flat = np.ndarray.flatten(trainY[0])

trainPredict_flat = np.ndarray.flatten(trainPredict[:,0])

In [ ]:
mean_absolute_percentage_error(trainY_flat, trainPredict_flat)

In [ ]:
testY_flat = np.ndarray.flatten(testY[0])

testPredict_flat = np.ndarray.flatten(testPredict[:,0])

In [ ]:
testY_flat = testY_flat[:334]
testPredict_flat = testPredict_flat[:334]

In [ ]:
y_originale = testY_flat
y_pred = testPredict_flat

In [ ]:
orizzonte=range(0,len(testY_flat))

In [ ]:
mean_absolute_percentage_error(y_originale,y_pred)

In [ ]:
fig= plt.figure(figsize=(35,15))

ax = fig.add_subplot()
ax.set_title('Forecast 334 giorni da 2018-01-01 a 2018-11-30', fontdict={'fontsize': 30, 'fontweight': 'medium'})

plt.plot(orizzonte, y_originale,c='black')
plt.plot(orizzonte,y_pred,c='blue')

In [ ]:
date_future = pd.date_range(start='2018-01-01', periods=334)

date_future = date_future.to_pydatetime()

In [ ]:
df_prev = pd.DataFrame(y_pred)
df_prev['Data'] = date_future

df_prev.columns=['RNN (2 hidden layer)','Data']

In [ ]:
df_prev = df_prev[['Data','RNN (2 hidden layer)']]

In [ ]:
df_prev.to_csv('/content/drive/My Drive/Streaming Data Management and time series/risultato_test_rnn_1passo.csv')

### Caricamento modello migliore 2 layer nascosti (1 passo in avanti)

In [ ]:
mod_2_hidden_best = load_model('/content/drive/My Drive/Streaming Data Management and time series/modelli_salvati/1passo/model_best_2_hidden_layer.h5')

In [ ]:
mod_2_hidden_best.summary()

In [ ]:
testX_def.shape

In [ ]:
pred_mod_best = mod_2_hidden_best.predict(testX_def)

pred_originale = scaler.inverse_transform(pred_mod_best)

pred_mod_best, pred_originale

In [ ]:
testX_def_iter = testX_def

In [ ]:
pred_future=[]

passati=np.ndarray.flatten(testX_def_iter)

for i in range(0,334):
  pred_norm = mod_2_hidden_best.predict(testX_def_iter)
  
  pred_originale = scaler.inverse_transform(pred_norm)
  pred_future.append(pred_originale[0][0])

  passati = np.append(passati,pred_norm)

  passati_succ = passati[i+1:]
  

  testX_def_iter = np.reshape(passati_succ, (1,1,len(passati_succ)))

In [ ]:
fig= plt.figure(figsize=(35,15))

ax = fig.add_subplot()
ax.set_title('Forecast 334 giorni da 2019-01-01 a 2019-11-30', fontdict={'fontsize': 30, 'fontweight': 'medium'})

plt.plot(orizzonte, pred_future,c='black')

### Previsioni future modello (1 passo in avanti)

In [ ]:
date_future = pd.date_range(start='2019-01-01', periods=334)

date_future = date_future.to_pydatetime()

In [ ]:
df_prev_future = pd.DataFrame(pred_future)

df_prev_future['Data'] = date_future

df_prev_future.columns=['RNN (1 passo)','Data']

In [ ]:
df_prev_future = df_prev_future[['Data','RNN (1 passo)']]

In [ ]:
df_prev_future.to_csv('/content/drive/My Drive/Streaming Data Management and time series/pred_finale_rnn_1passo.csv')